# Решение слегка упрощенной задачи от НТС

В данной задаче у всех заказов цена одинаковая, поэтому для уменьшения размерности задачи их можно агрегировать.

После агрегации видно, что в каждом из городов находится по три заказа в непересекающихся временных окнах.

Таким образом задачу можно свести к задаче описанной в ProblemDescription_RU, преообразовав каждый заказ к отдельному городу.

Это было сделано. Преобразованный файл ./tests/data/test05_nts_simple.xlsx.

Пример:

_Район 1_ преобразуется в _Район 1 w1_,  _Район 1 w2_,  _Район 1 w3_, где w1, w2, w3 - означают различные окна по времени.

In [1]:
import sys
import pandas as pd

sys.path.append("vehicle_routing/")

from data_reader import DataReader
from data_processor import DataProcessor
from data_writer import DataWriter
from linprog_maker import LinProgMaker

In [2]:
in_file = "tests/data/test05_nts_simple.xlsx"
out_file = "tests/data/output_test05_nts_simple.xlsx"

In [4]:
reader = DataReader(in_file)
reader.set_trips([1])
processor = DataProcessor(reader)
task = LinProgMaker(processor)
#task.solve()
writer = DataWriter(task, out_file)

## Данные

#### Заказы - покупатели

In [5]:
reader.read_orders()

,City,Price_per_unit,Deliver_from,Deliver_to,Demand
0,Район 1 w1,100,1,8,10046
1,Район 1 w2,100,9,16,6328
2,Район 1 w3,100,17,24,5867
3,Район 2 w1,100,1,8,5507
4,Район 2 w2,100,9,16,6130
5,Район 2 w3,100,17,24,22603
6,Район 3 w1,100,1,8,2404
7,Район 3 w2,100,9,16,7110
8,Район 3 w3,100,17,24,3603
9,Район 4 w1,100,1,8,11236


#### ТС - машины

In [6]:
reader.read_vehicles()

,CarName,Capacity,Price_per_km
0,1,20000,200
1,2,20000,200
2,3,20000,200
3,4,7500,200
4,5,15000,300
5,6,10000,250


#### Маршруты

In [7]:
reader.read_routes().head()

,From,To,Time,Distance
0,База,Район 1 w1,2,100
1,Район 2 w1,Район 1 w1,2,100
2,Район 3 w1,Район 1 w1,4,200
3,Район 4 w1,Район 1 w1,5,300
4,Район 5 w1,Район 1 w1,5,300


## Решение и комментарии

#### Таблица - Распределение маршрутов

In [9]:
#writer.make_vehicle_routing()

В этой таблице:

- TripNumber - номер поездки, но его порядок не должен согласовываться с временем отправления. 

- Delivered - доставлено продукта в City_to.

- Lack - сколько продукта все еще требуется доставить.

- DepartureTime - время отправки из вершины.

- ArrivalTime - время прибытия в вершину.

- Deliver_From, Deliver_To - ограничения по времени доставки продукта покупателю (для справки).

#### Таблица - Доставлено покупателям, начальный спрос, неудалось доставить

In [10]:
#writer.make_delivery()

#### Комментарии

- Остановил оптимизатор после 30 минут работы. 

- Ввденное руками ограничение на количество рейсов = 1 не помогло. 

- Слишком много вариантов решения. Нужно делать оганичения более жесткими.